# Limpieza y Transformación de Datos de Ofertas Inmobiliarias

Este notebook documenta el proceso de limpieza y transformación de datos para el conjunto de datos de ofertas inmobiliarias.
Se abordan los siguientes pasos:
1. Análisis y diagnósticos de anómalias
2. Identificación y tratamiento de valores faltantes.
3. Conversión de tipos de datos.
4. Eliminación de duplicados.
5. Tratamiento de outliers.
6. Transformación adicional de los datos según sea necesario.


### Importamos las librerias

In [2]:
import pandas as pd
import numpy as np

### Análisis y diagnósticos de anómalias 


In [3]:

# Cargar el dataset
file_path = '../data/raw/ofertas_inmobiliarias.csv'
df = pd.read_csv(file_path)

# Mostrar las primeras filas
df.head()



,Precio,Habitaciones,Baños,Parqueos,Sector,Condicion,Uso Actual,mt2,Terreno mt2,Piso/Nivel,Ascensor,Edificable,Agno Construccion,Planta Electrica,Seguridad 24 Horas,Control de Acceso,Piscina,Gimnasio
0,950.0,1,2.0,1,El Millón,N/D,Residencial,73.0,0.0,5,True,No,N/D,False,False,False,True,True
1,17000.0,4,4.5,5,Av. Anacaona,Segundo Uso,Residencial,611.0,0.0,0,True,No,2020,True,True,True,True,True
2,11000.0,3,4.5,4,Av. Anacaona,Segundo Uso,Residencial,598.0,598.0,0,True,No,N/D,True,True,True,True,True
3,11000.0,3,4.5,2,Av. Anacaona,Segundo Uso,Residencial,598.0,0.0,0,True,No,N/D,True,True,True,True,True
4,11000.0,4,4.5,4,Av. Anacaona,Nueva,Residencial,615.0,0.0,14,True,No,2023,True,True,True,True,False


In [4]:
df.shape #Tiene 1000 registros y 18 variables

(1000, 18)

In [5]:

# Revisión de valores faltantes y tipos de datos
missing_values = df.isnull().sum()
data_types = df.dtypes

missing_values #datos faltantes



Precio                0
Habitaciones          0
Baños                 0
Parqueos              0
Sector                0
Condicion             0
Uso Actual            0
mt2                   0
Terreno mt2           0
Piso/Nivel            0
Ascensor              0
Edificable            0
Agno Construccion     0
Planta Electrica      0
Seguridad 24 Horas    0
Control de Acceso     0
Piscina               0
Gimnasio              0
dtype: int64

In [6]:
data_types  #tipos de datos

Precio                float64
Habitaciones            int64
Baños                 float64
Parqueos                int64
Sector                 object
Condicion              object
Uso Actual             object
mt2                   float64
Terreno mt2           float64
Piso/Nivel              int64
Ascensor                 bool
Edificable             object
Agno Construccion      object
Planta Electrica         bool
Seguridad 24 Horas       bool
Control de Acceso        bool
Piscina                  bool
Gimnasio                 bool
dtype: object

In [7]:
df_filtered = df.select_dtypes(include=['bool', 'object'])
unicos = {}
for column in df_filtered.columns:
    unique_values = np.unique(df_filtered[column])
    unicos[column] = unique_values
    
unicos

{'Sector': array(['  Arroyo Hondo Viejo', ' 30 de mayo', ' Altos De Arroyo Hondo II',
        ' Altos de Arroyo Hondo III', ' Arroyo Manzano', ' Atala',
        ' Autopista Duarte', ' Av. Anacaona', ' Av. Cayetano Germosén',
        ' Av. Independencia', ' Av. Máximo Gómez',
        ' Av. República de Colombia', ' Bella Vista', ' Bella Vista Norte',
        ' Bella Vista Sur', ' Buenos Aires del Mirador',
        ' Centro de los Heroes', ' Ciudad Colonial', ' Ciudad Real',
        ' Ciudad Real II', ' Colinas de Arroyo Hondo II', ' Costa Azul',
        ' El Cacique', ' El Milloncito', ' El Millón', ' El Vergel',
        ' Enriquillo', ' Ensanche La Fé', ' Ensanche Naco',
        ' Ensanche Paraíso', ' Ensanche Quisqueya', ' Evaristo Morales',
        ' Gazcue', ' Jardines del Embajador', ' Julieta Morales',
        ' La Castellana', ' La Esperilla', ' La Julia',
        ' Las Colinas de Los Rios', ' Las Praderas', ' Los Cacicazgos',
        ' Los Jardines del Sur', ' Los Prados', ' Los

### Tratamiento de valores faltantes

Vemos que no hemos detectado ningun valor faltante ya que este no era tratado como NA sino como "N/D". Remplacemos y volvamos a verificar la cantidad de nulos

In [8]:
# Reemplazar 'N/D' con NaN para un mejor manejo
df.replace('N/D', pd.NA, inplace=True)
missing_values = df.isnull().sum()

missing_values

Precio                  0
Habitaciones            0
Baños                   0
Parqueos                0
Sector                  0
Condicion              31
Uso Actual             20
mt2                     0
Terreno mt2             0
Piso/Nivel              0
Ascensor                0
Edificable              0
Agno Construccion     743
Planta Electrica        0
Seguridad 24 Horas      0
Control de Acceso       0
Piscina                 0
Gimnasio                0
dtype: int64

Podemos observar que de 1000 registros, años de construccion tiene 743 faltantes lo que es un 74.3% del total

Igualmente condicion, uso actual poseen algunos valores faltantes pero en un porcentaje manejable


Imputemos la variable año de construccion con la siguiente estrategia: Agrupemos por sector y calculemos la media de años para imputar con este numero los valores faltantes

In [9]:
# Convertir la columna 'Agno Construccion' a tipo numérico
df['Agno Construccion'] = pd.to_numeric(df['Agno Construccion'], errors='coerce') #Transformamos a numeros

# Obtener la mediana general para usarla en caso de que algún grupo esté vacío
global_median = df['Agno Construccion'].median()



In [10]:

# Imputar valores faltantes por la mediana dentro de cada 'Sector'
def safe_median(x):
    if x.notna().sum() > 0:  # Si el grupo tiene al menos un valor no nulo
        return x.fillna(x.median())
    else:
        return x.fillna(global_median)  # Si el grupo está vacío, usar la mediana global

df['Agno Construccion'] = df.groupby('Sector')['Agno Construccion'].transform(safe_median)

np.unique(df["Agno Construccion"])

array([1978. , 1990. , 1992. , 2000. , 2002. , 2005. , 2007.5, 2008. ,
       2009. , 2010. , 2012. , 2013. , 2014. , 2015. , 2016. , 2017. ,
       2018. , 2019. , 2020. , 2021. , 2022. , 2022.5, 2023. , 2024. ])

Imputemos por la moda a las variables condicion y uso actual

In [11]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].apply(lambda col: col.fillna(col.mode()[0]))

np.unique(df["Condicion"])

array(['En Construcción', 'Nueva', 'Remodelada', 'Segundo Uso'],
      dtype=object)

In [12]:
np.unique(df["Uso Actual"])

array(['Comercial', 'Mixto', 'Residencial'], dtype=object)

In [13]:
#Volvemos a verificar la cantidad de nulos en nuestras variables
missing_values = df.isnull().sum()

missing_values

Precio                0
Habitaciones          0
Baños                 0
Parqueos              0
Sector                0
Condicion             0
Uso Actual            0
mt2                   0
Terreno mt2           0
Piso/Nivel            0
Ascensor              0
Edificable            0
Agno Construccion     0
Planta Electrica      0
Seguridad 24 Horas    0
Control de Acceso     0
Piscina               0
Gimnasio              0
dtype: int64

Las variables numericas tambien podrian presentar valores faltantes, por la naturaleza de la pagina web estos valores faltantes se representaban como 0. Reemplacemos estos 0 por NA

In [14]:
df.describe()

,Precio,Habitaciones,Baños,Parqueos,mt2,Terreno mt2,Piso/Nivel,Agno Construccion
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,2265.928183,2.099000,2.389500,1.760000,121.335000,14.346000,3.827000,2021.70400
std,9128.045584,0.870607,0.826573,0.718969,110.735284,60.545719,4.199131,3.08373
min,0.016667,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1978.00000
25%,1000.000000,1.000000,1.500000,1.000000,63.000000,0.000000,0.000000,2021.00000
50%,1300.000000,2.000000,2.500000,2.000000,107.500000,0.000000,3.000000,2022.50000
75%,2000.000000,3.000000,3.000000,2.000000,167.000000,0.000000,6.000000,2023.00000
max,200000.000000,10.000000,5.500000,6.000000,869.000000,854.000000,27.000000,2024.00000


Aca observamos anomalias en mt2, terreno mt2 y en Piso/Nivel. Donde estas variables no deberian ser nulas.

Verifiquemos cuantos hay con la categoria piso = 0. y verifiquemos si existe la categoria piso = 1. Para evitar confusiones de que el 0 represente primer nivel


In [15]:
print(len(df[df["Piso/Nivel"] == 1])) # Hay 16 ofertas que tienen como piso el primero

16


In [16]:
print(len(df[df["Piso/Nivel"] == 0])) # Hay 343 ofertas que tienen como piso el 0

343


Reemplacemos los 0 por NA en mt2, Terreno mt2 y Piso/nivel

In [18]:
df['mt2'] = df['mt2'].replace(0, np.nan) # Metros cuadrados

In [19]:
df['Terreno mt2'] = df['Terreno mt2'].replace(0, np.nan) # Terreno en mt2

In [17]:
df['Piso/Nivel'] = df['Piso/Nivel'].replace(0, np.nan) # Piso/nivel 

Verifiquemos los resultados:

In [21]:
#Cantidad de nulos
missing_values = df.isnull().sum()

missing_values

Precio                  0
Habitaciones            0
Baños                   0
Parqueos                0
Sector                  0
Condicion               0
Uso Actual              0
mt2                   190
Terreno mt2           892
Piso/Nivel            333
Ascensor                0
Edificable              0
Agno Construccion       0
Planta Electrica        0
Seguridad 24 Horas      0
Control de Acceso       0
Piscina                 0
Gimnasio                0
dtype: int64

Imputemos por la media segun sector a los metros cuadrados para rellenar los valores faltantes

In [22]:
df['mt2'] = df.groupby('Sector')['mt2'].transform(lambda x: x.fillna(x.mean()))

Consideraremos eliminar la variable Terreno mt2, esto es debido a que en investigaciones de la pagina web, o los vendedores no consideran relevante al vender inmuebles y no territorio, o los que si aparecen publicados tienen numeros coincidentes con el de metros cuadrados, por lo que crea redundancia y no agrega valor a posterior

Ej:
![image.png](img/Eje1.png)

In [23]:
df.drop('Terreno mt2', axis=1, inplace=True) #Eliminamos la variable "Terreno mt2"


### Remover duplicados

In [20]:
# Remover duplicados
df = df.drop_duplicates()
len(df)

974

Hemos eliminado 26 registros duplicados

### Manejo de outliers

In [62]:
df["Precio"].describe()

count       974.000000
mean       2287.674384
std        9247.497375
min           0.016667
25%        1000.000000
50%        1350.000000
75%        2000.000000
max      200000.000000
Name: Precio, dtype: float64

Vemos en el precio que la media es alrededor de 2300 dolares mensual, pero vemos algunas anomalias como puede ser 0.01 dolar  y un precio maximo de 200,000$ dolares mensual. 

Esto concuerda con algunas anomalias que hemos observado al hacer el analisis a la pagina web, donde algunos vendedores se confundian de moneda y publicaban ej: US$30,000/mes, y mas abajo en la observacion se veia la rectificacion a pesos. O tambien el error de tipeo donde se publica US$13,000/mes cuando realmente queria transmitir US$1,300/mes y este se observa la rectificacion tambien en la observacion.

count    974.000000
mean     122.146817
std      111.741590
min        0.000000
25%       63.000000
50%      108.000000
75%      169.000000
max      869.000000
Name: mt2, dtype: float64

In [3]:

# Manejo de outliers en 'Precio' y 'mt2' (limitando al percentil 1 y 99)
df['Precio'] = df['Precio'].clip(lower=df['Precio'].quantile(0.01), upper=df['Precio'].quantile(0.99))
df['mt2'] = df['mt2'].clip(lower=df['mt2'].quantile(0.01), upper=df['mt2'].quantile(0.99))

# Mostrar datos limpiados
df.head()


,Precio,Habitaciones,Baños,Parqueos,Sector,Condicion,Uso Actual,mt2,Terreno mt2,Piso/Nivel,Ascensor,Edificable,Agno Construccion,Planta Electrica,Seguridad 24 Horas,Control de Acceso,Piscina,Gimnasio
0,950.0,1,2.0,1,El Millón,Segundo Uso,Residencial,73.00,0.0,5,True,No,2024,False,False,False,True,True
1,10640.0,4,4.5,5,Av. Anacaona,Segundo Uso,Residencial,608.84,0.0,0,True,No,2020,True,True,True,True,True
2,10640.0,3,4.5,4,Av. Anacaona,Segundo Uso,Residencial,598.00,598.0,0,True,No,2024,True,True,True,True,True
3,10640.0,3,4.5,2,Av. Anacaona,Segundo Uso,Residencial,598.00,0.0,0,True,No,2024,True,True,True,True,True
4,10640.0,4,4.5,4,Av. Anacaona,Nueva,Residencial,608.84,0.0,14,True,No,2023,True,True,True,True,False


### Exportamos los datos ya procesados

In [4]:

# Guardar datos limpiados en un CSV
cleaned_csv_path = "../data/processed/cleaned_ofertas_inmobiliarias.csv"
df.to_csv(cleaned_csv_path, index=False)
cleaned_csv_path


'../data/processed/cleaned_ofertas_inmobiliarias.csv'